In [67]:
import pandas as pd
import numpy as np
import RNG

### Simulation parameters and helper functions

In [70]:
num_order = np.random.randint(low=5,high=11,size=1)[0]#  ~ Unif[1,11)
num_order

# unit of time: minuate
spawn_meant = 0.25
teleport_meant = 0.5
assemble_meant = 1.0
insp_meant = 1.5
pass_meant = 1.0

class ActionLogger(object):
    def __init__(self):
        self.clock = 0.0
        self.n = 0
        self.alog = []
        self.tlog = []

    def write_act(self,a,t):
        self.clock += t
        self.n += 1
        self.alog.append(a)
        self.tlog.append(self.clock)

### Station 1 - Wheelsets x 2

In [71]:
logger1 = ActionLogger()
for _ in range(num_order):
    for _ in range(2):
        # sourcing
        for _ in range(7):
            logger1.write_act("SpawnPart",RNG.Expon(spawn_meant,1))

        # manufacturing
        # (rim + tire) x 2
        for _ in range(2):
            logger1.write_act("TeleportPart",RNG.Expon(teleport_meant,1)) # rim
            logger1.write_act("TeleportPart",RNG.Expon(teleport_meant,1)) # tire
            logger1.write_act("Assemble",RNG.Expon(assemble_meant,1)) # rim + tire

        # (wheels + axle)
        logger1.write_act("TeleportPart",RNG.Expon(teleport_meant,1)) # axle
        logger1.write_act("Assemble",RNG.Expon(assemble_meant,1)) # axle + wheel 1
        logger1.write_act("Assemble",RNG.Expon(assemble_meant,1)) # axle + wheel 2
        
        # (axle + axle)
        logger1.write_act("TeleportPart",RNG.Expon(teleport_meant,1)) # connecting plate
        logger1.write_act("Assemble",RNG.Expon(assemble_meant,1)) # axle + axle

        # inspection
        logger1.write_act("Inspection",RNG.Expon(insp_meant,1))
    # pass wheelsets to downstream
    logger1.write_act("Pass",RNG.Expon(pass_meant,1))

### Station 2 - Chassis

In [72]:
upstream_parts_idx = np.where(np.array(logger1.alog)=="Pass")[0]
upstream_parts_time = np.array(logger1.tlog)[upstream_parts_idx]

logger2 = ActionLogger()
num_received_up = 0
num_base = 0
num_complete = 0

# first chassis base (long plate + long plate)
for _ in range(4):
    logger2.write_act("SpawnPart",RNG.Expon(spawn_meant,2))
logger2.write_act("TeleportPart",RNG.Expon(teleport_meant,2))
logger2.write_act("TeleportPart",RNG.Expon(teleport_meant,2))
logger2.write_act("Assemble",RNG.Expon(assemble_meant,2))
logger2.write_act("TeleportPart",RNG.Expon(teleport_meant,2))
logger2.write_act("Assemble",RNG.Expon(assemble_meant,2))
logger2.write_act("TeleportPart",RNG.Expon(teleport_meant,2))
logger2.write_act("Assemble",RNG.Expon(assemble_meant,2))
num_base += 1

while True:
    # need to receive wheelsets from upstream
    if logger2.clock < upstream_parts_time[num_received_up]:
        # keep making bases until getting wheelsets from upstream
        for _ in range(4):
            logger2.write_act("SpawnPart",RNG.Expon(spawn_meant,2))        
        logger2.write_act("TeleportPart",RNG.Expon(teleport_meant,2))
        logger2.write_act("TeleportPart",RNG.Expon(teleport_meant,2))
        logger2.write_act("Assemble",RNG.Expon(assemble_meant,2))
        logger2.write_act("TeleportPart",RNG.Expon(teleport_meant,2))
        logger2.write_act("Assemble",RNG.Expon(assemble_meant,2))
        logger2.write_act("TeleportPart",RNG.Expon(teleport_meant,2))
        logger2.write_act("Assemble",RNG.Expon(assemble_meant,2))
        num_base += 1
    else:
        num_received_up += 1
        for _ in range(5):
            logger2.write_act("SpawnPart",RNG.Expon(spawn_meant,2))

        # (wheelsets + base)
        logger2.write_act("TeleportPart",RNG.Expon(teleport_meant,2))
        logger2.write_act("Assemble",RNG.Expon(assemble_meant,2))
        logger2.write_act("TeleportPart",RNG.Expon(teleport_meant,2))
        logger2.write_act("Assemble",RNG.Expon(assemble_meant,2))        
        
        # (front bumper)
        logger2.write_act("TeleportPart",RNG.Expon(teleport_meant,2))
        logger2.write_act("TeleportPart",RNG.Expon(teleport_meant,2))
        logger2.write_act("Assemble",RNG.Expon(assemble_meant,2))
        logger2.write_act("TeleportPart",RNG.Expon(teleport_meant,2))
        logger2.write_act("Assemble",RNG.Expon(assemble_meant,2))

        # (engine bay)
        logger2.write_act("TeleportPart",RNG.Expon(teleport_meant,2))
        logger2.write_act("Assemble",RNG.Expon(assemble_meant,2))
        logger2.write_act("TeleportPart",RNG.Expon(teleport_meant,2))
        logger2.write_act("Assemble",RNG.Expon(assemble_meant,2))

        # assemble base and front module
        logger2.write_act("Assemble",RNG.Expon(assemble_meant,2))

        # inspection & pass chassis to downstream
        logger2.write_act("Inspection",RNG.Expon(insp_meant,2))
        logger2.write_act("Pass",RNG.Expon(pass_meant,2))
        num_complete += 1

    # completed all necessary chassis bases
    if num_base == num_order:
        break

# empty out chassis bases in inventory
while num_received_up < num_order:
    if logger2.clock < upstream_parts_time[num_received_up]:
        # wating wheelsets from upstream and complete chassis
        logger2.clock = upstream_parts_time[num_received_up]
    else:
        num_received_up += 1
        for _ in range(5):
            logger2.write_act("SpawnPart",RNG.Expon(spawn_meant,2))

        # (wheelsets + base)
        logger2.write_act("TeleportPart",RNG.Expon(teleport_meant,2))
        logger2.write_act("Assemble",RNG.Expon(assemble_meant,2))
        logger2.write_act("TeleportPart",RNG.Expon(teleport_meant,2))
        logger2.write_act("Assemble",RNG.Expon(assemble_meant,2))        
        
        # (front bumper)
        logger2.write_act("TeleportPart",RNG.Expon(teleport_meant,2))
        logger2.write_act("TeleportPart",RNG.Expon(teleport_meant,2))
        logger2.write_act("Assemble",RNG.Expon(assemble_meant,2))
        logger2.write_act("TeleportPart",RNG.Expon(teleport_meant,2))
        logger2.write_act("Assemble",RNG.Expon(assemble_meant,2))

        # (engine bay)
        logger2.write_act("TeleportPart",RNG.Expon(teleport_meant,2))
        logger2.write_act("Assemble",RNG.Expon(assemble_meant,2))
        logger2.write_act("TeleportPart",RNG.Expon(teleport_meant,2))
        logger2.write_act("Assemble",RNG.Expon(assemble_meant,2))

        # assemble base and front module
        logger2.write_act("Assemble",RNG.Expon(assemble_meant,2))

        # inspection & pass chassis to downstream
        logger2.write_act("Inspection",RNG.Expon(insp_meant,2))
        logger2.write_act("Pass",RNG.Expon(pass_meant,2))
        num_complete += 1

### Station 3 - Windshield & steering

In [73]:
upstream_parts_idx = np.where(np.array(logger2.alog)=="Pass")[0]
upstream_parts_time = np.array(logger2.tlog)[upstream_parts_idx]

logger3 = ActionLogger()
num_received_up = 0
num_steer = 0
num_complete = 0

# first steering
for _ in range(2):
    logger3.write_act("SpawnPart",RNG.Expon(spawn_meant,3))
logger3.write_act("TeleportPart",RNG.Expon(teleport_meant,3))
logger3.write_act("TeleportPart",RNG.Expon(teleport_meant,3))
logger3.write_act("Assemble",RNG.Expon(assemble_meant,3))
num_steer += 1

while True:
    # need to receive chassis from upstream
    if logger3.clock < upstream_parts_time[num_received_up]:
        # keep making front body frame until getting chassis from upstream
        for _ in range(2):
            logger3.write_act("SpawnPart",RNG.Expon(spawn_meant,3))
        logger3.write_act("TeleportPart",RNG.Expon(teleport_meant,3))
        logger3.write_act("TeleportPart",RNG.Expon(teleport_meant,3))
        logger3.write_act("Assemble",RNG.Expon(assemble_meant,3))
        num_steer += 1
    else:
        num_received_up += 1
        
        # windshield
        logger3.write_act("SpawnPart",RNG.Expon(spawn_meant,3))
        logger3.write_act("TeleportPart",RNG.Expon(teleport_meant,3))
        logger3.write_act("Assemble",RNG.Expon(assemble_meant,3))

        # (chassis + steering module)
        logger3.write_act("TeleportPart",RNG.Expon(teleport_meant,3))
        logger3.write_act("Assemble",RNG.Expon(assemble_meant,3))

        # inspection & pass to downstream
        logger3.write_act("Inspection",RNG.Expon(insp_meant,3))
        logger3.write_act("Pass",RNG.Expon(pass_meant,3))
        num_complete += 1

    # completed all necessary front body frames
    if num_steer == num_order:
        break

# empty out steering modules in inventory
while num_received_up < num_order:
    if logger3.clock < upstream_parts_time[num_received_up]:
        # waiting chassis coming from upstream
        logger3.clock = upstream_parts_time[num_received_up]
    else:
        num_received_up += 1
        
        # windshield
        logger3.write_act("SpawnPart",RNG.Expon(spawn_meant,3))
        logger3.write_act("TeleportPart",RNG.Expon(teleport_meant,3))
        logger3.write_act("Assemble",RNG.Expon(assemble_meant,3))

        # (chassis + steering module)
        logger3.write_act("TeleportPart",RNG.Expon(teleport_meant,3))
        logger3.write_act("Assemble",RNG.Expon(assemble_meant,3))

        # inspection & pass to downstream
        logger3.write_act("Inspection",RNG.Expon(insp_meant,3))
        logger3.write_act("Pass",RNG.Expon(pass_meant,3))
        num_complete += 1

### Station 4 - Body & ceiling

In [74]:
upstream_parts_idx = np.where(np.array(logger3.alog)=="Pass")[0]
upstream_parts_time = np.array(logger3.tlog)[upstream_parts_idx]

logger4 = ActionLogger()
num_received_up = 0
num_body = 0
nun_complete = 0

# first body frame
for _ in range(7):
    logger4.write_act("SpawnPart",RNG.Expon(spawn_meant,4))
logger4.write_act("TeleportPart",RNG.Expon(teleport_meant,4))
logger4.write_act("TeleportPart",RNG.Expon(teleport_meant,4))
logger4.write_act("Assemble",RNG.Expon(assemble_meant,4))
logger4.write_act("TeleportPart",RNG.Expon(teleport_meant,4))
logger4.write_act("Assemble",RNG.Expon(assemble_meant,4))
logger4.write_act("TeleportPart",RNG.Expon(teleport_meant,4))
logger4.write_act("Assemble",RNG.Expon(assemble_meant,4))
logger4.write_act("TeleportPart",RNG.Expon(teleport_meant,4))
logger4.write_act("Assemble",RNG.Expon(assemble_meant,4))
logger4.write_act("TeleportPart",RNG.Expon(teleport_meant,4))
logger4.write_act("Assemble",RNG.Expon(assemble_meant,4))
logger4.write_act("TeleportPart",RNG.Expon(teleport_meant,4))
logger4.write_act("Assemble",RNG.Expon(assemble_meant,4))
num_body += 1

while True:
    if logger4.clock < upstream_parts_time[num_received_up]:
        # keep making body frames until getting parts from upstream
        for _ in range(7):
            logger4.write_act("SpawnPart",RNG.Expon(spawn_meant,4))
        logger4.write_act("TeleportPart",RNG.Expon(teleport_meant,4))
        logger4.write_act("TeleportPart",RNG.Expon(teleport_meant,4))
        logger4.write_act("Assemble",RNG.Expon(assemble_meant,4))
        logger4.write_act("TeleportPart",RNG.Expon(teleport_meant,4))
        logger4.write_act("Assemble",RNG.Expon(assemble_meant,4))
        logger4.write_act("TeleportPart",RNG.Expon(teleport_meant,4))
        logger4.write_act("Assemble",RNG.Expon(assemble_meant,4))
        logger4.write_act("TeleportPart",RNG.Expon(teleport_meant,4))
        logger4.write_act("Assemble",RNG.Expon(assemble_meant,4))
        logger4.write_act("TeleportPart",RNG.Expon(teleport_meant,4))
        logger4.write_act("Assemble",RNG.Expon(assemble_meant,4))
        logger4.write_act("TeleportPart",RNG.Expon(teleport_meant,4))
        logger4.write_act("Assemble",RNG.Expon(assemble_meant,4))
        num_body += 1
    else:
        num_received_up += 1

        # ceiling
        logger4.write_act("SpawnPart",RNG.Expon(spawn_meant,4))
        logger4.write_act("TeleportPart",RNG.Expon(teleport_meant,4))
        logger4.write_act("Assemble",RNG.Expon(assemble_meant,4))

        # inspection & pass to downstream
        logger4.write_act("Inspection",RNG.Expon(insp_meant,4))
        logger4.write_act("Pass",RNG.Expon(pass_meant,4))
        num_complete += 1

    # completed all necessary body frames
    if num_body == num_order:
        break

# empty out body frames in inventory
while num_received_up < num_order:
    if logger4.clock < upstream_parts_time[num_received_up]:
        # waiting parts coming from upstream
        logger4.clock = upstream_parts_time[num_received_up]
    else:
        num_received_up += 1
        
        # ceiling
        logger4.write_act("SpawnPart",RNG.Expon(spawn_meant,4))
        logger4.write_act("TeleportPart",RNG.Expon(teleport_meant,4))
        logger4.write_act("Assemble",RNG.Expon(assemble_meant,4))

        # inspection & pass to downstream
        logger4.write_act("Inspection",RNG.Expon(insp_meant,4))
        logger4.write_act("Pass",RNG.Expon(pass_meant,4))
        num_complete += 1

In [84]:
# player,action,time_stamp
log_df1 = pd.DataFrame({'player':"player1", 'action':logger1.alog, 'time_stamp':logger1.tlog})
log_df2 = pd.DataFrame({'player':"player2", 'action':logger2.alog, 'time_stamp':logger2.tlog})
log_df3 = pd.DataFrame({'player':"player3", 'action':logger3.alog, 'time_stamp':logger3.tlog})
log_df4 = pd.DataFrame({'player':"player4", 'action':logger4.alog, 'time_stamp':logger4.tlog})
log_df = pd.concat([log_df1,log_df2,log_df3,log_df4])
log_df = log_df.sort_values(by=['time_stamp'],ascending=True)
log_df.to_csv('./sim_data/assembly_line_4players.csv',index=False)

In [83]:
num_order

8